In [1]:
import torch
print("Broj GPU-ova:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(f"{i}: {torch.cuda.get_device_name(i)}")


Broj GPU-ova: 4
0: Tesla V100-SXM2-16GB
1: Tesla V100-SXM2-16GB
2: Tesla V100-SXM2-16GB
3: Tesla V100-SXM2-16GB


# Model distilroberta-base-sst2-distilled

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer
from datasets import load_from_disk
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification

2025-10-28 09:37:27.605273: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-28 09:37:27.664614: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-28 09:37:29.233327: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [4]:
train_dataset = load_from_disk("tokenized/train")
val_dataset = load_from_disk("tokenized/val")
test_dataset = load_from_disk("tokenized/test")

train_dataset.set_format("torch")
val_dataset.set_format("torch")
test_dataset.set_format("torch")

In [ ]:
model_name = "azizbarank/distilroberta-base-sst2-distilled"

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  
    ignore_mismatched_sizes=True
)


config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [5]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
        "precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    }


In [7]:
training_args = TrainingArguments(
    output_dir="./distilroberta-base-sst2-distilled",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
metrics = trainer.evaluate(test_dataset)
print(metrics)


In [ ]:
trainer.save_model("./distilroberta-base-sst2-distilled")


### Dotreniravanje

In [ ]:
model_path = "./distilroberta-base-sst2"
model_distil_sst2 = AutoModelForSequenceClassification.from_pretrained(model_path)

In [7]:
continue_training_args = TrainingArguments(
    output_dir="./distilroberta-base-sst2-distilled",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100
)


In [8]:
trainer = Trainer(
    model=model_distil_sst2,
    args=continue_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_258211/2397596463.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.106500,0.143245,0.950453,0.950452,0.950480,0.950453
2,0.107700,0.141704,0.950822,0.950822,0.950822,0.950822


/opt/conda/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=12658, training_loss=0.10765477744187012, metrics={'train_runtime': 6067.3172, 'train_samples_per_second': 1068.013, 'train_steps_per_second': 2.086, 'total_flos': 2.1459639101832806e+17, 'train_loss': 0.10765477744187012, 'epoch': 2.0})

In [ ]:
trainer.save_model("./distilroberta-base-sst2-distilled")
tokenizer.save_pretrained("./distilroberta-base-sst2-distilled")